# Rapport - Devoir #2
## PHQ404
Maestracci Nathan - Romain Marcelin
## Introduction
Dans ce rapport, nous présentons une implémentation python pour construire itérativement un hamiltonien de N élecrons en suivant le modèle d'Heisenberg. On calcule ensuite les énergies de l'état fondamental et du premier état excité pour les systèmes de 2 à 20 électrons. On implémente ensuite l'algorithme epsilon pour calculer la convergence de la différence de ces deux énergies, quand N est poussé vers l'infini.

## Comment utiliser le programme

* Toutes les fonctions ont été compilées dans le fichier fonctions.py. Tous les autres fichiers devraient contenir, dans leurs premières lignes, import fonctions.py.

* Tous les résultats qui sont demandés explicitement ont un programme associé, qui sera mentionné dans la partie adéquate. Il suffit de lancer ce programme python pour obtenir les résultats.

## Hamiltonien

### Construction

La fonction `hamiltonien()` prend deux arguments en entrée : $N$ et $J$. $N$ est la dimension de la matrice Hamiltonien et J est une constante de couplage. La fonction commence par créer une matrice creuse H2 de dimension 4x4 qui représente le Hamiltonien de second ordre.

On peut ensuite multiplier cet hamiltonien sur tous les sites avec l'identité en généralisant la formule donnée dans l'énoncé:
$ H_N = \sum{I \otimes ... \otimes H \otimes ... \otimes I }  + \sum _p S^p \otimes S^p$

On y additionne les produit $S \otimes I \otimes S$ des matrices de Pauli avec l'identité, pour chaque ordre l'identité est d'ordre $2^{(k-2)}$ pour que le produit soit d'ordre $ 2^k$ comme le Hamiltonien.

Si $N$ est égal à 2, la fonction renvoie la matrice $H2$. Sinon, la fonction utilise une boucle pour calculer la somme des produits tensoriels pour des systèmes à $N$ niveaux. La matrice Hamiltonien est créée en utilisant la fonction $sp.kron$ (produit de Kronecker) qui calcule le produit tensoriel de deux matrices creuses.

En fin de compte, la fonction renvoie le produit de la matrice Hamiltonien $H$ avec la constante de couplage $J$, où $J$ est une mesure de l'interaction entre les électrons.

### Tests

On a écrit un programmee qui utilise la fonction `hamiltonien()` pour imprimer le hamiltonien d'ordre 3 et de constante de couplage 4. Pour y acceder, il suffit d'executer le fichier  `test_hamiltonien_3.py`

### Calcul

Nous avons également calculé à la main le Hamiltonien d'ordre 3 et de couplage $J = 4$ ainsi:

In [8]:
import numpy as np
import scipy.sparse as sp

Sx = sp.bsr_matrix([[0, 0.5], [0.5, 0]]) #On crée les matrices de Pauli
Sy = sp.bsr_matrix([[0, -0.5j], [0.5j, 0]])
Sz = sp.bsr_matrix([[0.5, 0], [0, -0.5]])

J=4 #Le couplage vaut 4
H2=(1/4)*sp.coo_matrix([[1,0,0,0],[0,-1,2,0],[0,2,-1,0],[0,0,0,1]]) #On utilise H2 donné dans l'énoncé

Sx_I_Sx=sp.kron(Sx, sp.kron(sp.eye(2), Sx)) #On calcule les produits S @ I @ S
Sy_I_Sy=sp.kron(Sy, sp.kron(sp.eye(2), Sy))
Sz_I_Sz=sp.kron(Sz, sp.kron(sp.eye(2), Sz))

H2_I=sp.kron(H2,sp.eye(2)) #Les produits H2 @ I et I @ H2 
I_H2=sp.kron(sp.eye(2),H2)

H3=J*(H2_I+I_H2+Sx_I_Sx+Sy_I_Sy+Sz_I_Sz) #On additionne le tout

print(H3.toarray().real) #On obtient H3

[[ 3.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  2.  0.  2.  0.  0.  0.]
 [ 0.  2. -1.  0.  2.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  2.  2.  0.]
 [ 0.  2.  2.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  2.  0. -1.  2.  0.]
 [ 0.  0.  0.  2.  0.  2. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  3.]]


On retrouve le même Hamiltonien en éxecutant le programme `test_hamiltonien_3.py `

## Energies

### Construction
Avec ce programme pour calculer les hamiltoniens en main, on peut calculer les énergies de ce dernier : Ce sont les valeurs propres du Hamiltonien. La fonction  `energie()` (dans le fichier `fonctions.py`) permet de calculer les deux premières valeurs propres non dégénérées d'une matrice creuse. 
On utilise la fonction `scipy.sparse.linalg.eigsh(which='SA)` qui calcule les valeur propres avec les plus petites valeurs algébriques, correspondant donc au énergies du hamiltonien concerné.
Pour minimiser le nombre de valeurs propres à calculer (à chaque valeur propre supplémentaire le temps augmente drastiquement) on calcule d'abord 2 valeurs, et si elles sont différentes on recommence en en calculant une de plus. Une tolérance de 1/10000 est utilisée, pour prendre en compte la non analycité de la fonction `eigsh()`.

### Résultats
Pour obtenir les 2 premières valeurs propres distinctes des hamiltoniens d'ordre 2 à 20, on peut utiliser la fonction `imprimer_energies.py`. Comme ce programme prend un certain temps, le fichier `data.txt` dans lequel ces energies sont stockées est déjà créé et disponible dans ce répositoire.

## Epsilon

## Construction

On construit comme dans les notes de cours l'algorithme epsilon, qui prend une liste python en entrée, et donne en sortie une prédiction pour la limite de la suite quand $N \rightarrow \infty$. L'algorithme est également dans le fichier `fonctions.py`. On a simplement stoppé l'algorithme quand il divise par zéro: on aura de toute façon pas mieux que la précision machine.

### Tests

#### Série de Gregory

La série de Grégory, qui est liée au développement de Taylor de l'arctangente en $x = 1$, tend vers $\pi$, assez lentement.
On utilise le fichier `test_epsilon_gregory.py` pour imprimer, d'abord, comme comparaison, le millième terme de la série de grégory (~3.1405) et les 5 premiers termes utiles de l'algorithme epsilon (dont le dernier est ~3.141593). On voit que la convergence avec l'algorithme epsilon est bien plus rapide.

#### Energies $E_0/N$ et $E_1 - E_0$

Nous avons également testé la fonction `epsilon()` sur la suite des $E_0 / N$ dans le programme `test_epsilon_E0N.py`. Celui ci calcule les termes $E_0/N$, en fait une suite, l'utilise en entrée du programme epsilon et trace un graphique, en utilisant une barre horizontale pour dénoter la valeur que retourne la fonction `epsilon()`.
Dans le même programme, on crée deux autres listes, `Liste_paire` et `Liste_impaire` qui sont respectivement les élements paires et impaires de la suite `Liste_test` qui compile les valeurs $E_0/N$
Les trois graphiques sont tracés simultannément, avec trois limites de convergence

On répète l'exacte même processus pour la suite $E_1-E_0$ en prenant les valeurs du même fichier texte `data.txt`. Le fichier python qui execute cette tâche et affiche les graphes est `test_epsilon_E0E1`.

On peut voir sur ces graphiques que les valeurs de convergences sont proches, mais généralement la meilleure est celle qui compile toutes les valeurs de la série.